# # Task 3 Music Recommendation
### Harsh Jain

In [37]:
import numpy as np
import pandas as pd
from typing import List, Dict
from tokenize import tokenize, untokenize, NUMBER, STRING, NAME, OP

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
path='songdata.csv'

In [40]:
df = pd.read_csv(path,quoting=0 )
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [41]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [42]:
df['text'] = df['text'].str.replace(r'\n', '')
tf = TfidfVectorizer(analyzer='word', stop_words='english')

C:\Users\anany\AppData\Local\Temp/ipykernel_12532/57601187.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r'\n', '')


In [43]:
tf

TfidfVectorizer(stop_words='english')

In [44]:
lyrmatrix = tf.fit_transform(df['text'].values.astype('U'))

In [45]:
cs = cosine_similarity(lyrmatrix)
similarities={}

In [46]:
for i in range(len(cs)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cs[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[df['song'].iloc[i]] = [(cs[i][x], df['song'][x], df['artist'][x]) for x in similar_indices][1:]

In [47]:
class Recommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [48]:
recs= Recommender(similarities)

In [49]:
rec = {
    "song": df['song'].iloc[16],
    "number_songs": 4
}

In [50]:
recs.recommend(rec)

The 4 recommended songs for She's Alright are:
Number 1:
Ain't It Funny by Loretta Lynn with 0.28 similarity score
--------------------
Number 2:
Not That Funny by Fleetwood Mac with 0.243 similarity score
--------------------
Number 3:
I've Got A Funny Feeling by George Strait with 0.231 similarity score
--------------------
Number 4:
At This Time by Pat Benatar with 0.217 similarity score
--------------------
